In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import string
import json

In [2]:
url = 'https://www.omscentral.com/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
# soup.find_all('table')

In [18]:
data = soup.select_one('#__NEXT_DATA__')

In [13]:
soup = BeautifulSoup(data, 'html.parser') 
res = soup.find('script') 
json_object = json.loads(res.contents[0])

In [21]:
data = soup.select_one('#__NEXT_DATA__')
courses = json.loads(data.contents[0])['props']['pageProps']['courses']

In [44]:
c_reviews = pd.json_normalize(courses[0]['reviews'])
c_reviews

,created,difficulty,id,rating,workload
0,2022-08-14T03:39:14Z,1,I85e2pBlQ8Ku7mLFcPZjYV,1.0,10.0
1,2022-08-05T03:03:15Z,3,LZcH8OQfhbE1IkMdNFr0io,5.0,20.0
2,2022-05-02T00:57:07Z,1,external-import-d40c1689-5d3d-4ad8-be25-e737cf...,4.0,4.0
3,2022-04-29T21:03:32Z,2,external-import-6ba6c430-bee8-4bbf-bbee-e5b532...,4.0,10.0
4,2022-04-24T16:55:50Z,2,external-import-bca644f7-f218-4d33-8b18-cb78bb...,5.0,11.0
...,...,...,...,...,...
414,2015-11-05T06:00:00Z,2,external-import-84f2e3bd-b098-46ac-a09f-26064e...,NaN,10.0
415,2015-11-05T06:00:00Z,2,external-import-a7da8e47-b6a9-4550-833e-eacf04...,NaN,10.0
416,2015-11-05T06:00:00Z,2,external-import-ac721080-2d01-4e61-bfb0-c8d17c...,NaN,10.0
417,2015-11-05T06:00:00Z,3,external-import-b2585673-b3be-41ce-a9b3-99e421...,NaN,10.0


In [48]:
reviews = pd.DataFrame()

for i in range(len(courses)):
    course = courses[i]['name']
    c_reviews = pd.json_normalize(courses[i]['reviews'])
    c_reviews["Course"] = course
    reviews = pd.concat([reviews, c_reviews])

In [55]:
pd.json_normalize(courses[0]['reviews'])

,created,difficulty,id,rating,workload
0,2022-08-14T03:39:14Z,1,I85e2pBlQ8Ku7mLFcPZjYV,1.0,10.0
1,2022-08-05T03:03:15Z,3,LZcH8OQfhbE1IkMdNFr0io,5.0,20.0
2,2022-05-02T00:57:07Z,1,external-import-d40c1689-5d3d-4ad8-be25-e737cf...,4.0,4.0
3,2022-04-29T21:03:32Z,2,external-import-6ba6c430-bee8-4bbf-bbee-e5b532...,4.0,10.0
4,2022-04-24T16:55:50Z,2,external-import-bca644f7-f218-4d33-8b18-cb78bb...,5.0,11.0
...,...,...,...,...,...
414,2015-11-05T06:00:00Z,2,external-import-84f2e3bd-b098-46ac-a09f-26064e...,NaN,10.0
415,2015-11-05T06:00:00Z,2,external-import-a7da8e47-b6a9-4550-833e-eacf04...,NaN,10.0
416,2015-11-05T06:00:00Z,2,external-import-ac721080-2d01-4e61-bfb0-c8d17c...,NaN,10.0
417,2015-11-05T06:00:00Z,3,external-import-b2585673-b3be-41ce-a9b3-99e421...,NaN,10.0


In [57]:
agg = reviews.groupby(["Course"]).agg({'difficulty':np.nanmean,'workload':np.nanmean,'rating':np.nanmean,'id':'count'})

In [58]:
agg = agg.reset_index()

,difficulty,workload,rating,id
Course,,,,
"AI, Ethics & Society",1.700000,6.040000,2.660000,50
Advanced Internet Systems & Applications,3.166667,13.166667,3.000000,6
Advanced Malware Analysis,3.571429,16.142857,4.571429,7
Advanced Operating Systems,4.157025,17.173554,4.116279,121
Applied Analytics Practicum,2.250000,13.500000,4.250000,4
...,...,...,...,...
Software Architecture & Design,2.603175,11.558511,3.127389,189
Software Development Process,2.301948,9.043434,3.601695,308
Systems Issues in Cloud Computing,4.444444,30.555556,4.666667,9


In [70]:
import plotly.express as px

X="difficulty"
Y="rating"
SIZE='workload'
color='id'

fig = px.scatter(agg, 
                 x="difficulty", 
                 y="rating",
                 size="workload", 
                 color="id",
                 hover_name=agg.index, 
                 size_max=50,
                 color_continuous_scale=px.colors.sequential.Viridis,
                 title="OMSCS Course Reviews - {} v/s {}".format(X.title(), Y.title())
                )
fig.add_hline(y=3)
fig.add_vline(x=3)
fig.update_layout(yaxis_range=[0,5.5],xaxis_range=[0,5])
# fig.add_vline(x=df_metrics["Avg. Delta PPSF"].mean())
fig.show()

In [73]:
import streamlit as st

In [72]:
st.title('OMSCS Reviews Visualization')

  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19567 sha256=f65cb57dff1bdc13eb430d6ea5636ff38b97199b9d1093fb076e217ecbf83d79
  Stored in directory: c:\users\steve\appdata\local\pip\cache\wheels\19\09\72\3eb74d236bb48bd0f3c6c3c83e4e0c5bbfcbcad7c6c3539db8
Successfully built validators
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
